In [1]:
# baseline 0.95214

In [2]:
# Logit Tf-Idf 6 features

In [3]:
# from Correct time-aware cross-validation scheme

In [4]:
# Import libraries and set desired options
import numpy as np
import pandas as pd
#from scipy.sparse import hstack
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
# A helper function for writing predictions to a file
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [7]:
train_df = pd.read_csv('train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv('test_sessions.csv',
                      index_col='session_id')

# Convert time1, ..., time10 columns to datetime type
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# Sort the data by time
train_df = train_df.sort_values(by='time1')

# Look at the first rows of the training set
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


In [8]:
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites].fillna(0).astype('int').to_csv('train_sessions_text.txt', 
                                               sep=' ', index=None, header=None)
test_df[sites].fillna(0).astype('int').to_csv('test_sessions_text.txt', 
                                              sep=' ', index=None, header=None)

In [9]:
!head -5 train_sessions_text.txt

56 55 0 0 0 0 0 0 0 0
56 55 56 55 0 0 0 0 0 0
946 946 951 946 946 945 948 784 949 946
945 948 949 948 945 946 947 945 946 946
947 950 948 947 950 952 946 951 946 947


In [10]:
%%time
cv = CountVectorizer(ngram_range=(1, 3), max_features=50000)
with open('train_sessions_text.txt') as inp_train_file:
    X_train = cv.fit_transform(inp_train_file)
with open('test_sessions_text.txt') as inp_test_file:
    X_test = cv.transform(inp_test_file)
X_train.shape, X_test.shape

Wall time: 14.5 s


In [11]:
y_train = train_df['target'].astype('int')

In [12]:
time_split = TimeSeriesSplit(n_splits=10)

In [13]:
[(el[0].shape, el[1].shape) for el in time_split.split(X_train)]

[((23051,), (23051,)),
 ((46102,), (23051,)),
 ((69153,), (23051,)),
 ((92204,), (23051,)),
 ((115255,), (23051,)),
 ((138306,), (23051,)),
 ((161357,), (23051,)),
 ((184408,), (23051,)),
 ((207459,), (23051,)),
 ((230510,), (23051,))]

In [14]:
logit = LogisticRegression(C=1, random_state=17)

In [15]:
#%%time

#cv_scores = cross_val_score(logit, X_train, y_train, cv=time_split, 
#                            scoring='roc_auc', n_jobs=-1) # hangs with n_jobs > 1, and locally this runs much faster

In [16]:
#cv_scores, cv_scores.mean()

In [17]:
#logit.fit(X_train, y_train)

In [18]:
#logit_test_pred = logit.predict_proba(X_test)[:, 1]
#write_to_submission_file(logit_test_pred, 'subm1.csv') # 0.91288

In [19]:
def add_time_features(df, X_sparse):
    hour = df['time1'].apply(lambda ts: ts.hour)
    morning = ((hour >= 7) & (hour <= 11)).astype('int')
    day = ((hour >= 12) & (hour <= 18)).astype('int')
    evening = ((hour >= 19) & (hour <= 23)).astype('int')
    night = ((hour >= 0) & (hour <= 6)).astype('int')
    X = hstack([X_sparse, morning.values.reshape(-1, 1), 
                day.values.reshape(-1, 1), evening.values.reshape(-1, 1), 
                night.values.reshape(-1, 1)])
    return X

In [20]:
%%time
X_train_new = add_time_features(train_df.fillna(0), X_train)
X_test_new = add_time_features(test_df.fillna(0), X_test)

Wall time: 1min 41s


In [21]:
X_train_new.shape, X_test_new.shape

((253561, 50004), (82797, 50004))

In [22]:
#%%time
#cv_scores = cross_val_score(logit, X_train_new, y_train, cv=time_split, 
#                            scoring='roc_auc', n_jobs=-1) # hangs with n_jobs > 1, and locally this runs much faster

In [23]:
#cv_scores, cv_scores.mean()

In [24]:
#logit.fit(X_train_new, y_train)

In [25]:
#logit_test_pred2 = logit.predict_proba(X_test_new)[:, 1]
#write_to_submission_file(logit_test_pred2, 'subm2.csv') # 0.93843


In [26]:
#c_values = np.logspace(-2, 2, 10)

#logit_grid_searcher = GridSearchCV(estimator=logit, param_grid={'C': c_values},
#                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [27]:
#%%time
#logit_grid_searcher.fit(X_train_new, y_train)

In [28]:
#logit_grid_searcher.best_score_, logit_grid_searcher.best_params_

In [29]:
#logit_test_pred3 = logit_grid_searcher.predict_proba(X_test_new)[:, 1]
#write_to_submission_file(logit_test_pred3, 'subm3.csv') # 0.94242

In [30]:
#GridSearchCV?

In [31]:
def add_day_features(df, X_sparse):
    day = df['time1'].apply(lambda ts: ts.weekday())
    weekend = ((day >= 5)).astype('int')
    weekday = ((day < 5)).astype('int')
    X = hstack([X_sparse, weekend.values.reshape(-1, 1), 
                weekday.values.reshape(-1, 1)])
    return X

In [32]:
%%time
X_train_new1 = add_time_features(train_df.fillna(0), X_train_new)
X_test_new1 = add_time_features(test_df.fillna(0), X_test_new)

Wall time: 1min 37s


In [33]:
#X_train_new1.shape, X_test_new1.shape

In [34]:
#%%time
#cv_scores = cross_val_score(logit, X_train_new1, y_train, cv=time_split, 
#                            scoring='roc_auc', n_jobs=-1) # hangs with n_jobs > 1, and locally this runs much faster

In [35]:
#cv_scores, cv_scores.mean()
# 0.9164082065409461

In [36]:
#%%time
#logit_grid_searcher.fit(X_train_new1, y_train)

In [37]:
#logit_grid_searcher.best_score_, logit_grid_searcher.best_params_
# (0.9183503264075926, {'C': 0.21544346900318834})

In [38]:
#logit_test_pred4 = logit_grid_searcher.predict_proba(X_test_new1)[:, 1]
#write_to_submission_file(logit_test_pred4, 'subm_self1.csv')

In [39]:
def label_unique_site_sum(row):
    s = set()
    for i in sites:
        if row[i] != 0:
            s.add(row[i])
            pass
        pass
    return sum(s)

In [40]:
def label_unique_site(row):
    s = set()
    for i in sites:
        if row[i] != 0:
            s.add(row[i])
            pass
        pass
    return len(s)

In [41]:
def label_unique_site_sum(row):
    s = set()
    for i in sites:
        if row[i] != 0:
            s.add(row[i])
            pass
        pass
    return sum(s)

In [42]:
def label_unique_site(row):
    s = set()
    for i in sites:
        if row[i] != 0:
            s.add(row[i])
            pass
        pass
    return len(s)

In [ ]:
### 7

In [ ]:
#### 8

In [40]:
### no more use the original data nummber

In [43]:
#train_df.head()

In [44]:
alice_web_statistic = {}
alice_web_count_statistic = {}

def getalice_feature(row):
    s = set()
    for i in sites:
        webid = row[i]
        s.add(webid)
        if webid in alice_web_statistic:
            alice_web_statistic[webid] = alice_web_statistic[webid] + 1
        else:
            alice_web_statistic[webid] = 1
            pass
        pass
    webcount = len(s)
    if webcount in alice_web_count_statistic:
        alice_web_count_statistic[webcount] = alice_web_count_statistic[webcount] + 1
    else:
        alice_web_count_statistic[webcount] = 1
        pass
    pass

#train_df[train_df['target'] == 1][sites].fillna(0).astype('int').apply(lambda row: getalice_feature(row),axis=1)

In [45]:
#alice_web_count_statistic

In [46]:
#alice_web_statistic

In [47]:
#alice_web_count = [(k, alice_web_count_statistic[k]) for k in sorted(alice_web_count_statistic, key=alice_web_count_statistic.get, reverse=True)][: 5]

In [48]:
#alice_love_web = [(k, alice_web_statistic[k]) for k in sorted(alice_web_statistic, key=alice_web_statistic.get, reverse=True)][: 10]

In [49]:
#alice_web_statistic[0]

In [50]:
def match_alice_love_web(row):
    count = 0
    for i in sites:
        if row[i] in alice_love_web:
            count += 1
            pass
        pass
    return count > 5

In [51]:
def match_alice_web_count(row):
    s = set()
    for i in sites:
        if row[i] != 0:
            s.add(row[i])
            pass
        pass
    return sum(s) in alice_web_count

In [52]:
def add_features8(df, X_sparse):
    hour = df['time1'].apply(lambda ts: ts.hour)
    morning = ((hour >= 7) & (hour <= 11)).astype('int')
    day = ((hour >= 12) & (hour <= 18)).astype('int')
    evening = ((hour >= 19) & (hour <= 23)).astype('int')
    night = ((hour >= 0) & (hour <= 6)).astype('int')
    
    dayinweek = df['time1'].apply(lambda ts: ts.weekday())
    weekend = ((dayinweek >= 5)).astype('int')
    weekday = ((dayinweek < 5)).astype('int')
    
    web_count_match = df.apply (lambda row: match_alice_web_count (row),axis=1).astype('int')
    
    web_type = df.apply (lambda row: match_alice_love_web (row),axis=1)
   
    
    X = hstack([X_sparse, morning.values.reshape(-1, 1), 
                day.values.reshape(-1, 1), evening.values.reshape(-1, 1), 
                night.values.reshape(-1, 1)
                , weekend.values.reshape(-1, 1)
                , weekday.values.reshape(-1, 1)
                , web_count_match.values.reshape(-1, 1)
                , web_type.values.reshape(-1, 1)
               ])
    return X

In [53]:
#%%time
#X_train_new8 = add_features8(train_df.fillna(0), X_train)
#X_test_new8 = add_features8(test_df.fillna(0), X_test)

In [54]:
#%%time
#cv_scores = cross_val_score(fastlogist, X_train_new8, y_train, cv=time_split, 
#                            scoring='scoring ', n_jobs=2) # hangs with n_jobs > 1, and locally this runs much faster

In [55]:
#cv_scores, cv_scores.mean()
# 0.914169880747776

In [56]:
#%%time
#fastlogist.fit(X_train_new8, y_train)

In [57]:
#logit_test_pred8 = fastlogist.predict_proba(X_test_new8)[:, 1]
#write_to_submission_file(logit_test_pred8, 'subm_self8.csv')

In [58]:
#%%time
#logit_grid_searcher.fit(X_train_new8, y_train)

In [59]:
#logit_grid_searcher.best_score_, logit_grid_searcher.best_params_
# (0.9081530648461433, {'C': 0.21544346900318834})

In [60]:
### 9

In [61]:

alice_web_count_score = {}

count = 0
for key, val in alice_web_count_statistic.items():
    count += val
    pass
for key, val in alice_web_count_statistic.items():
    alice_web_count_score[key] = val * 1.0 / count
    pass

In [62]:

alice_web_score = {}
count = 0

for key, val in alice_web_statistic.items():
    count += val
    pass
for key, val in alice_web_statistic.items():
    alice_web_score[key] = val * 1.0 / count
    pass

In [63]:
def get_web_score(row):
    score = 0
    for i in sites:
        if row[i] != 0:
            if row[i] in alice_web_count_score:
                score += alice_web_count_score[row[i]]
            pass
        pass
    return score
    pass

In [64]:
def get_web_count_score(row):
    count = 0
    for i in sites:
        if row[i] != 0:
            count += 1
            pass
        pass
    if count in alice_web_count_score:
        return alice_web_count_score[count]
    else:
        return 0
    pass

In [65]:
## 10

In [66]:
#np.linspace(0.1, 1, 10)

In [67]:
#logit_grid_searcher3 = GridSearchCV(estimator=logit, param_grid={'C': np.linspace(0.1, 1, 10)},
#                                  scoring='accuracy', n_jobs=-1, cv=time_split, verbose=1)

In [68]:
#logit_grid_searcher3.fit(X_train_new10, y_train)

In [69]:
#logit_grid_searcher3.best_score_, logit_grid_searcher3.best_params_

In [70]:
#logit_test_pred10 = logit_grid_searcher3.predict_proba(X_test_new10)[:, 1]
#write_to_submission_file(logit_test_pred10, 'subm_self10.csv')

In [71]:
#logit_grid_searcher4 = GridSearchCV(estimator=logit, param_grid={'C': c_values},
#                                  scoring='accuracy', n_jobs=-1, cv=time_split, verbose=1)

In [72]:
#logit_grid_searcher4.fit(X_train_new10, y_train)

In [73]:
#logit_grid_searcher4.best_score_, logit_grid_searcher4.best_params_

In [74]:
#logit_test_pred11 = logit_grid_searcher4.predict_proba(X_test_new10)[:, 1]
#write_to_submission_file(logit_test_pred11, 'subm_self11.csv')

In [75]:
#np.logspace(-2, 2, 10)

In [76]:
#np.logspace(-3, 1, 10)

In [77]:
#c_values = np.logspace(-3, 1, 10)

#logit_grid_searcher_1 = GridSearchCV(estimator=logit, param_grid={'C': c_values},
#                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [78]:
#logit_grid_searcher_1.fit(X_train_new10, y_train)

In [79]:
#logit_grid_searcher_1.best_score_, logit_grid_searcher_1.best_params_
# 0.913374049587434

In [80]:
#%%time
#logit_grid_searcher_1.fit(X_train_new1, y_train)

In [81]:
#logit_grid_searcher_1.best_score_, logit_grid_searcher_1.best_params_
# 0.9181526261645014

In [82]:
#np.linspace(0.15, 0.25, 10)

In [83]:
#logit_grid_searcher_2 = GridSearchCV(estimator=logit, param_grid={'C': np.linspace(0.15, 0.25, 10)},
#                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [84]:
#%%time
#logit_grid_searcher_2.fit(X_train_new1, y_train)

In [85]:
#logit_grid_searcher_2.best_score_, logit_grid_searcher_2.best_params_
# (0.9183729937045094, {'C': 0.22777777777777777})

In [86]:
#logit_grid_searcher_3 = GridSearchCV(estimator=logit, param_grid={'C': np.linspace(0.22, 0.6, 20)},
#                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [87]:
#%%time
#logit_grid_searcher_3.fit(X_train_new1, y_train)

In [88]:
#logit_grid_searcher_3.best_score_, logit_grid_searcher_3.best_params_
# (0.9183505761017501, {'C': 0.24})

In [89]:
#logit_grid_searcher_4 = GridSearchCV(estimator=logit, param_grid={'C': np.linspace(0.22, 0.25, 10)},
#                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [90]:
#%%time
#logit_grid_searcher_4.fit(X_train_new1, y_train)

In [91]:
#logit_grid_searcher_4.best_score_, logit_grid_searcher_4.best_params_

In [92]:
#logit_grid_searcher_5 = GridSearchCV(estimator=logit, param_grid={'C': np.linspace(0.223, 0.229, 10)},
#                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [93]:
#%%time
#logit_grid_searcher_5.fit(X_train_new1, y_train)

In [94]:
#logit_grid_searcher_5.best_score_, logit_grid_searcher_5.best_params_

In [95]:
#logit_test_pred12 = logit_grid_searcher_5.predict_proba(X_test_new1)[:, 1]
#write_to_submission_file(logit_test_pred12, 'subm_self12.csv')

In [96]:
### 11

In [97]:
### feature day_of_week, web_unique, time_diff1-10, total_time, 

In [98]:
### inspire by https://www.kaggle.com/adityaecdrid/initial-eda and https://www.kaggle.com/hakeydotcom/additional-time-features-and-logit

In [99]:
### optional stop hour

In [100]:
def get_web_unique(row):
    s = set()
    for i in sites:
        if row[i] != 0:
            s.add(row[i])
            pass
        pass
    return len(s)

In [101]:
def get_total_time(row):
    if len(times) <= 1:
        return 0
    start = row[times[0]]
    end = start
    for i in times:
        if row[i] == 0:
            break
            pass
        end = row[i]
        pass
    return (end - start).total_seconds() #/ np.timedelta64(1, 's')

In [102]:
def get_diff_time(row, f, t):
    if len(times) < f or len(times) < t:
        return 0
    if row[times[f]] == 0 or row[times[t]] == 0:
        return 0
    return (row[times[t]] - row[times[f]]).total_seconds() #/ np.timedelta64(1, 's')

In [103]:
def add_features_11(df, X_sparse):

    total_time = df.apply(lambda row: get_total_time(row), axis=1)
    
    # ((hour >= 12) & (hour <= 18)).astype('int')
    #total_time_5 = (total_time <= 300).astype('int')
    #total_time_10 = ((total_time > 300) & (total_time <= 600)).astype('int')
    #total_time_15 = ((total_time > 600) & (total_time <= 900)).astype('int')
    #total_time_20 = ((total_time > 900) & (total_time <= 1200)).astype('int')
    #total_time_25 = ((total_time > 1200) & (total_time <= 1500)).astype('int')
    #total_time_30 = ((total_time > 1500) & (total_time <= 1800)).astype('int')
    
    #diff_time1 = df.apply(lambda row: get_diff_time(row, 0, 1), axis=1)
    #diff_time2 = df.apply(lambda row: get_diff_time(row, 1, 2), axis=1)
    #diff_time3 = df.apply(lambda row: get_diff_time(row, 2, 3), axis=1)
    #diff_time4 = df.apply(lambda row: get_diff_time(row, 3, 4), axis=1)
    #diff_time5 = df.apply(lambda row: get_diff_time(row, 4, 5), axis=1)
    #diff_time6 = df.apply(lambda row: get_diff_time(row, 5, 6), axis=1)
    #diff_time7 = df.apply(lambda row: get_diff_time(row, 6, 7), axis=1)
    #diff_time8 = df.apply(lambda row: get_diff_time(row, 7, 8), axis=1)
    #diff_time9 = df.apply(lambda row: get_diff_time(row, 8, 9), axis=1)
    
    scaler = StandardScaler() 
    #hour = scaler.fit_transform(hour.values.reshape(-1, 1))
    #dayinweek = scaler.fit_transform(dayinweek.values.reshape(-1, 1))
    #web_unique = scaler.fit_transform(web_unique.values.reshape(-1, 1))
    #total_time = scaler.fit_transform(total_time.values.reshape(-1, 1))
    #diff_time1 = scaler.fit_transform(diff_time1.values.reshape(-1, 1))
    #diff_time2 = scaler.fit_transform(diff_time2.values.reshape(-1, 1))
    #diff_time3 = scaler.fit_transform(diff_time3.values.reshape(-1, 1))
    #diff_time4 = scaler.fit_transform(diff_time4.values.reshape(-1, 1))
    #diff_time5 = scaler.fit_transform(diff_time5.values.reshape(-1, 1))
    #diff_time6 = scaler.fit_transform(diff_time6.values.reshape(-1, 1))
    #diff_time7 = scaler.fit_transform(diff_time7.values.reshape(-1, 1))
    #diff_time8 = scaler.fit_transform(diff_time8.values.reshape(-1, 1))
    #diff_time9 = scaler.fit_transform(diff_time9.values.reshape(-1, 1))
    
    X = hstack([X_sparse

                , total_time.values.reshape(-1, 1)
               ])
    return X

In [104]:
## alice 

In [105]:
#alice_web_statistic

In [106]:
#alice_web_statistic = {}
#alice_web_count_statistic = {}

def getwebid_feature(row, outdict):
    for i in sites:
        webid = row[i]
        if webid in outdict:
            outdict[webid] = outdict[webid] + 1
        else:
            outdict[webid] = 1
            pass
        pass
    pass

#train_df[train_df['target'] == 1][sites].fillna(0).astype('int').apply(lambda row: getalice_feature(row),axis=1)

In [107]:
alice_webid = {}
alice_webid_df = train_df[train_df['target'] == 1][sites].fillna(0).astype('int').apply(lambda row: getwebid_feature(row, alice_webid),axis=1)

In [108]:
nonalice_webid = {}
nonalice_webid_df = train_df[train_df['target'] == 0][sites].fillna(0).astype('int').apply(lambda row: getwebid_feature(row, nonalice_webid),axis=1)

In [109]:
len(alice_webid)

1054

In [110]:
len(nonalice_webid)

41412

In [111]:
alice_webid_set = alice_webid.keys()

In [112]:
nonalice_webid_set = nonalice_webid.keys()

In [113]:
print(type(alice_webid_set), type(nonalice_webid_set))

<class 'dict_keys'> <class 'dict_keys'>


In [114]:
only_alice = alice_webid_set - nonalice_webid_set

In [115]:
#only_alice


In [180]:
sorted_by_value = sorted(alice_webid.items(), key=lambda kv: kv[1], reverse=True)

In [185]:
#sorted_by_value[0][1]

1382

In [186]:
#sorted_by_value
alice_top10 = set()
alice_top10_total = 0
for i in range(0, 10):
    alice_top10_total += sorted_by_value[i][1]
    alice_top10.add(sorted_by_value[i][0])
    pass

In [187]:
alice_top10_total

8348

In [188]:
alice_top10

{21, 22, 29, 75, 76, 77, 80, 81, 82, 879}

In [182]:
alice_total = 0

for key, val in alice_webid.items():
    alice_total +=val

In [183]:
alice_total

22970

In [201]:
nonalice_top = sorted(nonalice_webid.items(), key=lambda kv: kv[1], reverse=True)

In [203]:
#nonalice_top

In [ ]:
nonalice_top

In [206]:
nonalice_top10 = set()
nonalice_top10_total = 0
for i in range(0, 11):
    if nonalice_top[i][0] != 0:
        nonalice_top10_total += nonalice_top[i][1]
        nonalice_top10.add(nonalice_top[i][0])
    pass

In [207]:
nonalice_top10

{21, 22, 23, 29, 52, 167, 778, 780, 782, 812}

In [208]:
nonalice_top10_total

597536

In [209]:
nonalice_total = 0

for key, val in nonalice_webid.items():
    nonalice_total +=val

In [210]:
nonalice_total

2512640

In [211]:
def mathNonAliceTop10(row):
    for i in sites:
        if row[i] in nonalice_top10:
            return True
        pass
    return False

In [189]:
def mathAliceTop10(row):
    for i in sites:
        if row[i] in alice_top10:
            return True
        pass
    return False

In [190]:
def matchOnlyAlice(row):
    for i in sites:
        if row[i] in only_alice:
            return True
        pass
    return False

In [220]:
def get_web_count(row):
    count = 0
    for i in sites:
        if row[i] != 0:
            count += 1
            pass
        pass
    return count

In [221]:
def add_features_lastone(df, X_sparse):
    #dayinweek = df['time1'].apply(lambda ts: ts.isoweekday())
    matchalicewebid = df.apply(lambda row: matchOnlyAlice(row), axis=1).astype('int')
    matchalicetop10 = df.apply(lambda row: mathAliceTop10(row), axis=1).astype('int')
    matchnonalicetop10 = df.apply(lambda row: mathNonAliceTop10(row), axis=1).astype('int')
    #hour = df['time1'].apply(lambda ts: ts.hour)
    web_unique = df.apply(lambda row: get_web_unique(row), axis=1)
    web_count = df.apply(lambda row: get_web_count(row), axis=1)
    
    scaler = StandardScaler() 
    #dayinmonth = scaler.fit_transform(dayinmonth.values.reshape(-1, 1))
    #hour = scaler.fit_transform(hour.values.reshape(-1, 1))
    #dayinweek = scaler.fit_transform(dayinweek.values.reshape(-1, 1))
    web_unique = scaler.fit_transform(web_unique.values.reshape(-1, 1))
    web_count = scaler.fit_transform(web_count.values.reshape(-1, 1))
    
    X = hstack([X_sparse
                #, dayinweek.values.reshape(-1, 1)
                #, dayinweek[:, :]
                , matchalicewebid.values.reshape(-1, 1)
                , matchalicetop10.values.reshape(-1, 1)
                , matchnonalicetop10.values.reshape(-1, 1)
                #, diff_time4[:, :]
                #, diff_time5[:, :]
                #, diff_time6[:, :]
                #, diff_time7[:, :]
                #, diff_time8[:, :]
                #, diff_time9[:, :]
                #, hour.values.reshape(-1, 1)
                #, hour[:, :]
                #, total_time_30.values.reshape(-1, 1)
                #, total_time.values.reshape(-1, 1)
                , web_unique[:, :]
                , web_count[:, :]
               ])
    return X

In [154]:
new_train_df = train_df.fillna(0)
new_test_df = test_df.fillna(0)

In [222]:
%%time
X_train_lastone = add_features_lastone(new_train_df, X_train_new1)
X_test_lastone = add_features_lastone(new_test_df, X_test_new1)

d:\program files\python37\lib\site-packages\sklearn\utils\validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
d:\program files\python37\lib\site-packages\sklearn\utils\validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
d:\program files\python37\lib\site-packages\sklearn\utils\validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
d:\program files\python37\lib\site-packages\sklearn\utils\validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Wall time: 2min 16s


d:\program files\python37\lib\site-packages\sklearn\utils\validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
d:\program files\python37\lib\site-packages\sklearn\utils\validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
d:\program files\python37\lib\site-packages\sklearn\utils\validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
d:\program files\python37\lib\site-packages\sklearn\utils\validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [223]:
logit_lastone = LogisticRegression(C=1, random_state=17)

In [224]:
%%time
cv_scores = cross_val_score(logit_lastone, X_train_lastone, y_train, cv=time_split, 
                            scoring='roc_auc', n_jobs=-1) # hangs with n_jobs > 1, and locally this runs much faster

Wall time: 55.4 s


In [225]:
cv_scores, cv_scores.mean()

(array([0.89759358, 0.77349503, 0.94366368, 0.98645955, 0.91852612,
        0.96931893, 0.96733199, 0.94371164, 0.95489104, 0.94501184]),
 0.9300003419280986)

In [227]:
c_values_lastone = np.logspace(-2, 2, 10)

logit_grid_searcher_lastone = GridSearchCV(estimator=logit_lastone, param_grid={'C': c_values_lastone},
                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [228]:
%%time
logit_grid_searcher_lastone.fit(X_train_lastone, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   46.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 10.7min finished
d:\program files\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Wall time: 11min 12s


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
       error_score='raise-deprecating',
       estimator=LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'C': array([1.00000e-02, 2.78256e-02, 7.74264e-02, 2.15443e-01, 5.99484e-01,
       1.66810e+00, 4.64159e+00, 1.29155e+01, 3.59381e+01, 1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [229]:
logit_grid_searcher_lastone.best_score_, logit_grid_searcher_lastone.best_params_

(0.9308925830271153, {'C': 0.5994842503189409})

In [199]:
logit_test_pred_lastone = logit_grid_searcher_lastone.predict_proba(X_test_lastone)[:, 1]
write_to_submission_file(logit_test_pred_lastone, 'subm_lastone.csv') # 

In [134]:
%%time
logit_lastone.fit(X_train_lastone, y_train)

d:\program files\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Wall time: 17.2 s


LogisticRegression(C=0.21544346900318834, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=42,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [135]:
logit_test_pred = logit_lastone.predict_proba(X_test_lastone)[:, 1]
write_to_submission_file(logit_test_pred, 'subm_lastone_1.csv') # 0.93843

In [178]:
#total_time = train_df.fillna(0).apply(lambda row: get_total_time(row), axis=1)

In [179]:
#total_time.head()

In [180]:
#total_time.iloc[1].total_seconds()

In [181]:
# # 0.9164082065409461

In [182]:
# # (0.9183503264075926, {'C': 0.21544346900318834})

In [183]:
## 12

In [188]:
%%time
X_train_new12 = add_features_11(train_df.fillna(0), X_train_new1)
X_test_new12 = add_features_11(test_df.fillna(0), X_test_new1)

Wall time: 2min 43s


In [291]:
# solver='liblinear'
logit12 = LogisticRegression(C=1, random_state=17, solver='liblinear')

In [292]:
%%time
cv_scores = cross_val_score(logit12, X_train_new12, y_train, cv=time_split, 
                            scoring='roc_auc', n_jobs=-1) # hangs with n_jobs > 1, and locally this runs much faster

Wall time: 59.6 s


In [293]:
cv_scores, cv_scores.mean()
# 0.9165846708050015 for 5, 10, 15, 20, 25, 30 total time
# 0.9176728004288149 for total time no op

(array([0.87785413, 0.75905071, 0.92989561, 0.97860492, 0.90468275,
        0.94581241, 0.96286484, 0.92797542, 0.94919461, 0.9407926 ]),
 0.9176728004288149)

In [294]:
logit.fit(X_train_new12, y_train)

d:\program files\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python37\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [295]:
logit_test_pred12_1 = logit.predict_proba(X_test_new12)[:, 1]
write_to_submission_file(logit_test_pred12_1, 'subm12_1.csv') # 0.93843

In [192]:
c_values12 = np.logspace(-2, 2, 10)

logit_grid_searcher12 = GridSearchCV(estimator=logit, param_grid={'C': c_values12},
                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [193]:
%%time
logit_grid_searcher12.fit(X_train_new12, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:  1.4min
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:  8.5min finished
d:\program files\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Wall time: 9min 1s


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
       error_score='raise-deprecating',
       estimator=LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=5,
       param_grid={'C': array([1.00000e-02, 2.78256e-02, 7.74264e-02, 2.15443e-01, 5.99484e-01,
       1.66810e+00, 4.64159e+00, 1.29155e+01, 3.59381e+01, 1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [194]:
logit_grid_searcher12.best_score_, logit_grid_searcher12.best_params_

(0.9187219397390473, {'C': 0.21544346900318834})

In [195]:
logit_test_pred12 = logit_grid_searcher12.predict_proba(X_test_new12)[:, 1]
write_to_submission_file(logit_test_pred12, 'subm12.csv') # 

In [ ]:
## 13

In [238]:
new_train_df = train_df.fillna(0)

In [248]:
new_test_df = test_df.fillna(0)

In [246]:
#new_train_df['time1'].iloc[0]
# Timestamp('2013-01-12 08:05:57')

Timestamp('2013-01-12 08:05:57')

In [252]:
#new_train_df['time1'].iloc[0].day

12

In [251]:
#new_train_df['time1'].iloc[0]

Timestamp('2013-01-12 08:05:57')

Object `Timestamp` not found.


In [196]:
def get_total_time_minute(row):
    if len(times) <= 1:
        return 0
    start = row[times[0]]
    end = start
    for i in times:
        if row[i] == 0:
            break
            pass
        end = row[i]
        pass
    return ((end - start).total_seconds() + 59) // 60 #/ np.timedelta64(1, 's')

In [234]:
def get_day_in_month(dt):
    return dt['time1'].date.day

In [333]:
def add_features_13(df, X_sparse):
    dayinweek = df['time1'].apply(lambda ts: ts.isoweekday())
    #dayinmonth = df['time1'].apply(lambda ts: ts.day)
    web_unique = df.apply(lambda row: get_web_unique(row), axis=1)
    total_time = df.apply(lambda row: get_total_time_minute(row), axis=1)
    hour = df['time1'].apply(lambda ts: ts.hour)
    
    dayinweek_1 = ((dayinweek == 1)).astype('int')
    dayinweek_3 = ((dayinweek == 3)).astype('int')
    #dayinweek_4_5 = ((dayinweek == 4) | (dayinweek == 5)).astype('int')
    #dayinweek_other = ((dayinweek != 4) & (dayinweek != 5) & (dayinweek != 1) & (dayinweek != 2)).astype('int')
    
    hourin_8 = ((hour == 8)).astype('int')
    hourin_12_15 = ((hour == 14) | (hour == 15)).astype('int')
    hourin_16_17_18 = ((hour == 16) | (hour == 17) | (hour == 18)).astype('int')
    #hourin_other = ((dayinweek != 12) & (dayinweek != 13) & (dayinweek != 16) & (dayinweek != 17) & (dayinweek != 18)).astype('int')
    
    total_time_5 = ((total_time <= 10)).astype('int')
    total_time_10 = ((total_time <= 10) & (total_time > 10)).astype('int')

    scaler = StandardScaler() 
    #dayinmonth = scaler.fit_transform(dayinmonth.values.reshape(-1, 1))
    #hour = scaler.fit_transform(hour.values.reshape(-1, 1))
    #dayinweek = scaler.fit_transform(dayinweek.values.reshape(-1, 1))

    
    X = hstack([X_sparse
                #, dayinweek.values.reshape(-1, 1)
                #, hour.values.reshape(-1, 1)
                #, web_unique.values.reshape(-1, 1)
                , dayinweek_1.values.reshape(-1, 1)
                , dayinweek_3.values.reshape(-1, 1)
                #, dayinweek_other.values.reshape(-1, 1)
                
                , hourin_8.values.reshape(-1, 1)
                , hourin_12_15.values.reshape(-1, 1)
                , hourin_16_17_18.values.reshape(-1, 1)
                
                , total_time_5.values.reshape(-1, 1)
                , total_time_10.values.reshape(-1, 1)
                #, hourin_other.values.reshape(-1, 1)
                #, total_time.values.reshape(-1, 1)
                #, web_unique[:, :]
                #, total_time[:, :]
                #, dayinweek[:, :]
                #, diff_time1[:, :]
                #, diff_time2[:, :]
                #, diff_time3[:, :]
                #, diff_time4[:, :]
                #, diff_time5[:, :]
                #, diff_time6[:, :]
                #, diff_time7[:, :]
                #, diff_time8[:, :]
                #, diff_time9[:, :]
                #, hour.values.reshape(-1, 1)
                #, hour[:, :]
                #, total_time_5.values.reshape(-1, 1)
                #, total_time_10.values.reshape(-1, 1)
                #, total_time_15.values.reshape(-1, 1)
                #, total_time_20.values.reshape(-1, 1)
                #, total_time_25.values.reshape(-1, 1)
                #, total_time_30.values.reshape(-1, 1)
                #, total_time.values.reshape(-1, 1)
                #, diff_time1.values.reshape(-1, 1)
                #, diff_time2.values.reshape(-1, 1)
                #, diff_time3.values.reshape(-1, 1)
                #, diff_time4.values.reshape(-1, 1)
                #, diff_time5.values.reshape(-1, 1)
                #, diff_time6.values.reshape(-1, 1)
                #, diff_time7.values.reshape(-1, 1)
                #, diff_time8.values.reshape(-1, 1)
                #, diff_time9.values.reshape(-1, 1)
                #, dayinmonth.values.reshape(-1, 1)
                #, dayinmonth[:, :]
               ])
    return X

In [334]:
%%time
X_train_new13 = add_features_13(new_train_df, X_train)
X_test_new13 = add_features_13(new_test_df, X_test)

Wall time: 1min 56s


In [328]:
logit13 = LogisticRegression(C=0.21544346900318834, random_state=42)

In [335]:
%%time
cv_scores = cross_val_score(logit, X_train_new13, y_train, cv=time_split, 
                            scoring='roc_auc', n_jobs=-1) # hangs with n_jobs > 1, and locally this runs much faster

Wall time: 40.1 s


In [336]:
cv_scores, cv_scores.mean()
# 0.9183928162548293
# 0.9212810466948731 use logit13

(array([0.85419456, 0.75904635, 0.97217055, 0.97229386, 0.91461613,
        0.97553383, 0.89916641, 0.94487496, 0.87322648, 0.97470356]),
 0.9139826692565496)

In [313]:
%%time
logit13.fit(X_train_new13, y_train)

d:\program files\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Wall time: 22.3 s


LogisticRegression(C=0.21544346900318834, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=42,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [314]:
logit_test_pred13_1 = logit13.predict_proba(X_test_new13)[:, 1]
write_to_submission_file(logit_test_pred13_1, 'subm13_1.csv') # 

In [301]:
c_values13 = np.logspace(-2, 2, 10)

logit_grid_searcher13 = GridSearchCV(estimator=logit, param_grid={'C': c_values13},
                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [302]:
%%time
logit_grid_searcher13.fit(X_train_new13, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   48.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  9.8min finished


Wall time: 10min 8s


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
       error_score='raise-deprecating',
       estimator=LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'C': array([1.00000e-02, 2.78256e-02, 7.74264e-02, 2.15443e-01, 5.99484e-01,
       1.66810e+00, 4.64159e+00, 1.29155e+01, 3.59381e+01, 1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [303]:
logit_grid_searcher12.best_score_, logit_grid_searcher12.best_params_
# (0.9187219397390473, {'C': 0.21544346900318834})

(0.9187219397390473, {'C': 0.21544346900318834})

In [270]:
logit14 = LogisticRegression(C=1, random_state=42)

In [271]:
c_values14 = np.logspace(-2, 2, 20)

logit_grid_searcher14 = GridSearchCV(estimator=logit14, param_grid={'C': c_values14},
                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [272]:
%%time
logit_grid_searcher14.fit(X_train_new13, y_train)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   42.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 16.3min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed: 20.2min finished
d:\program files\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Wall time: 20min 30s


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
       error_score='raise-deprecating',
       estimator=LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'C': array([1.00000e-02, 1.62378e-02, 2.63665e-02, 4.28133e-02, 6.95193e-02,
       1.12884e-01, 1.83298e-01, 2.97635e-01, 4.83293e-01, 7.84760e-01,
       1.27427e+00, 2.06914e+00, 3.35982e+00, 5.45559e+00, 8.85867e+00,
       1.43845e+01, 2.33572e+01, 3.79269e+01, 6.15848e+01, 1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [274]:
logit_grid_searcher14.best_score_, logit_grid_searcher14.best_params_, logit_grid_searcher14.best_estimator_, logit_grid_searcher14.best_params_

(0.913085746373354,
 {'C': 0.18329807108324356},
 LogisticRegression(C=0.18329807108324356, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2', random_state=42,
           solver='warn', tol=0.0001, verbose=0, warm_start=False),
 {'C': 0.18329807108324356})

In [275]:
logit_test_pred14 = logit_grid_searcher14.predict_proba(X_test_new13)[:, 1]
write_to_submission_file(logit_test_pred14, 'subm13.csv') # 

In [ ]:
## random forest

In [284]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [285]:
rf = RandomForestRegressor(n_estimators=10)

In [286]:
%%time
cv_scores = cross_val_score(rf, X_train_new13, y_train, cv=time_split, 
                            scoring='roc_auc', n_jobs=-1) # hangs with n_jobs > 1, and locally this runs much faster

Wall time: 22min 12s


(array([0.66537805, 0.59268359, 0.73395755, 0.86501889, 0.64571057,
        0.71369666, 0.88386354, 0.61923017, 0.73335726, 0.70858517]),
 0.7161481442263696)

ValueError: X has 50026 features per sample; expecting 50009